In [5]:
import os

In [6]:
%pwd

'C:\\Users\\91781\\PycharmProjects\\Kideney_Disease_Classification_MLflow_DVC\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'C:\\Users\\91781\\PycharmProjects\\Kideney_Disease_Classification_MLflow_DVC'

In [6]:
import sys
print(sys.executable)


C:\Users\91781\anaconda3\envs\kidney\python.exe


In [16]:
import dagshub
dagshub.init(repo_owner='tanishak0000007777',
             repo_name='Kidney_Disease_Classification_MLflow_DVC',
             mlflow=True)

import mlflow
print("✅ Connected to DagsHub successfully!")


Accessing as tanishak0000007777

Initialized MLflow to track repo "tanishak0000007777/Kidney_Disease_Classification_MLflow_DVC"

Repository tanishak0000007777/Kidney_Disease_Classification_MLflow_DVC initialized!

✅ Connected to DagsHub successfully!


In [1]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPUs:", tf.config.list_physical_devices('GPU'))



TF version: 2.20.0
Num GPUs Available: 0
GPUs: []


In [4]:
import tensorflow as tf



In [9]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [25]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [26]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/kdc_dataset"),
            mlflow_uri="https://dagshub.com/tanishak0000007777/Kidney_Disease_Classification_MLflow_DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [33]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

    # Create validation data generator
    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.30)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    # Load saved model
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    # Evaluate the model
    def evaluation(self):
        print("🔍 Loading model...")
        self.model = self.load_model(self.config.path_of_model)
        print("✅ Model loaded successfully.")

        print("📦 Preparing validation data...")
        self._valid_generator()

        print("🧪 Evaluating model...")
        self.score = self.model.evaluate(self.valid_generator)
        print(f"✅ Evaluation complete — Loss: {self.score[0]:.4f}, Accuracy: {self.score[1]:.4f}")

        self.save_score()

    # Save scores locally
    def save_score(self):
        scores = {"loss": float(self.score[0]), "accuracy": float(self.score[1])}
        save_json(path=Path("scores.json"), data=scores)
        print("💾 Scores saved to scores.json")

    # Log results to DagsHub (MLflow)
    def log_into_mlflow(self):
        print("🚀 Logging metrics & model to MLflow (DagsHub)...")
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # ✅ Create a custom run name (e.g. "VGG16 Evaluation Run")
        with mlflow.start_run(run_name="VGG16 Evaluation Run"):
            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            # ✅ Save model locally first
            import os
            local_model_path = "artifacts/evaluated_model"
            os.makedirs(local_model_path, exist_ok=True)
            model_save_path = os.path.join(local_model_path, "VGG16Model.h5")
            self.model.save(model_save_path)

            # ✅ Log as artifact (model)
            mlflow.log_artifact(model_save_path, artifact_path="VGG16Model")

            # ✅ Add custom tags for better organization
            mlflow.set_tag("model_name", "VGG16")
            mlflow.set_tag("phase", "evaluation")
            mlflow.set_tag("framework", "TensorFlow")

        print("🎯 Metrics & 'VGG16Model' successfully logged to DagsHub!")



In [32]:
if __name__ == "__main__":
    try:
        config = ConfigurationManager()
        eval_config = config.get_evaluation_config()
        evaluation = Evaluation(eval_config)

        evaluation.evaluation()
        evaluation.log_into_mlflow()

    except Exception as e:
        print(f"❌ Error during evaluation: {e}")
        raise e

🔍 Loading model...


✅ Model loaded successfully.
📦 Preparing validation data...
Found 2207 images belonging to 2 classes.
🧪 Evaluating model...
138/138 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.6760 - loss: 2.0119
✅ Evaluation complete — Loss: 2.0119, Accuracy: 0.6760
💾 Scores saved to scores.json
🚀 Logging metrics to MLflow (DagsHub)...


🏃 View run blushing-chimp-196 at: https://dagshub.com/tanishak0000007777/Kidney_Disease_Classification_MLflow_DVC.mlflow/#/experiments/0/runs/13e023735d434324a4e3e14248803732
🧪 View experiment at: https://dagshub.com/tanishak0000007777/Kidney_Disease_Classification_MLflow_DVC.mlflow/#/experiments/0
🎯 Metrics & model successfully logged to DagsHub!
